In [ ]:
from requests import get
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import time

In [ ]:
comment_list = [] #Master list for all comments
thread_title_list = []
comment_author_list =[]
date_list = []
thread_count = 0 
comment_count=0

for x in range(1,100):
    
    page_count = x
    
    #Threads with most number of replies
    url = 'https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/index{}.html?sort=replycount&order=desc&daysprune=-1'.format(x)
    if x == 1:
        url = 'https://forums.hardwarezone.com.sg/eat-drink-man-woman-16/?sort=replycount&order=desc&daysprune=-1'
        
    response = get(url)
    html_soup = BeautifulSoup(response.text, 'html.parser')
    
    
    thread_href_list = html_soup.find_all('a', id=re.compile('thread_title'))
    
    if x == 1:
        del thread_href_list[:5] #Remove stickied posts on first EDMW page
    
    for thread_href in thread_href_list:
       
        thread_link = 'https://forums.hardwarezone.com.sg' + thread_href['href']
        response2 = get(thread_link)
        soup = BeautifulSoup(response2.text, 'html.parser')
        
        try:
            number_of_pages = soup.find('div', attrs={'class':'pagination'}).span.text.replace('Page 1 of ','') #Last page of the thread
            
        except:
            print('Error')
            print(number_of_pages)
            continue
            
        else:
            thread_title = soup.find('h2', {'class':'header-gray'}).text
            thread_count += 1
            
        
    
        
        for page_number in range(1,int(number_of_pages)+1):
            
            thread_page_link =  'https://forums.hardwarezone.com.sg' + thread_href['href'].strip('.html') + '-'+ str(page_number) + '.html'
            response3 = get(thread_page_link)
            page_soup = BeautifulSoup(response3.text, 'html.parser')
            
            comment_containers = page_soup.find_all('div', {'class':'post-wrapper'})
            
    
            for container in comment_containers:
                
                
                date = container.find('td').text.strip('\n\t\r ')
                if 'Today' in date:
                    date = date.replace('Today', '29-04-2019')
                if 'Yesterday' in date:
                    date = date.replace('Yesterday', '28-04-2019')
                    
                if '2018' not in date and '2019' not in date:
                    continue
                else:
                    comment_count+=1
                    #Capture HTML comment section 
                    comment = container.find('div',attrs = {'class':'post_message'})
                    if comment:
                        comment = comment.text
                    else:
                        comment = '' # Comments that have no text (eg. only pictures)
                    
                    if container.find('div',{'class':'quote'}): # If comment contains quote,
                        for quote in container.find_all('div',{'class':'quote'}):
                            comment = comment.replace(quote.text, '') # Remove quote
                            
           
                    #Capture HTML author section
                    comment_author = container.find('a',{'class':'bigusername'})
                    if comment_author:
                        comment_author = comment_author.text
                    else: # guest users will not have a 'bigusername' class, so just call all guest users 'EDMW Guest User'
                        comment_author = 'EDMW Guest User'
              
            
                    thread_title_list.append(thread_title)
                    comment_author_list.append(comment_author)
                    date_list.append(date)
                    comment_list.append(comment)
           
         

            print('Page / Thread / Comment Count : {} / {} / {}'.format(page_count,thread_count,comment_count), end ='\r') #Print thread/comment count at after every page                  
            
        
            
print('\nTotal Thread Count: {}\nTotal Comment Count: {}'.format(thread_count,comment_count))


In [ ]:
len(comment_list)

In [ ]:
len(thread_title_list)

In [ ]:
len(comment_author_list)

In [ ]:
df = pd.DataFrame({'Thread Title':thread_title_list[:-] ,'Comment Author':comment_author_list[:],'Comment':comment_list[:], 'Date':date_list[:]}) #Create pandas object
df.head(20)

In [ ]:
#Remove emoji before saving to csv file or will have encode error
# https://zhuanlan.zhihu.com/p/41213713

def remove_emoji(text):

        
    myre = re.compile(u'['
                          u'\U0001F300-\U0001F64F'
                          u'\U0001F680-\U0001F6FF'
                          u'\u2600-\u2B55'
                          u'\u23cf'
                          u'\u23e9'
                          u'\u231a'
                          u'\u3030'
                          u'\ufe0f'
                          u'\ude18'
                          u'\udd29'
                          u'\ud83d'
                          u'\ud83e'
                          u'\ude14'
                          u'\udc99'
                          u'\ude02'
                          u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                            u'\U00010000-\U0010ffff'
                           u'\U0001F1E0-\U0001F1FF'  # flags (iOS)
                           u'\U00002702-\U000027B0]+',     
                          re.UNICODE)
  
    text = myre.sub(' ', text)
    return text
        


df['Comment'] = df['Comment'].apply(remove_emoji) 

In [ ]:
export_count = 0

for x in range(df['Comment'].size):
    try:
        export_count += 1
        print(export_count, end = '\r')
        df.iloc[[x]].to_csv('~/Desktop/Project_Forum/edmw_data_raw.csv', mode='a', index = False, header=False)
    except:
        print('error')
        continue